# Drag AMPERE

Ivestigate density and AMPERE data

## Datasets

Use the normalized datasets. 
- satdrag_database_grace_B.hdf5
- satdrag_database_grace_C.hdf5
- satdrag_database_grace_CHAMP.hdf5

In [1]:
#print all output in a cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import netCDF4 as nc


# for converting to
# geomagnetic coord
import aacgmv2



In [3]:
# min and max lat
lmin = 30
lmax = 90

# magnetic min and max
mlmin = 40
mlmax = 90

# date range (for AMPERE data)
dmin = '2009-01-01'
dmax = '2013-01-01'

# satellite to use
sat = 'B'
data_f=f"D:/data/SatDensities/satdrag_database_grace_{sat}.hdf5"

# ampere file
# single file to find indexes of closest encounter
data_amp = "D:/data/AMPERE/amp_capfit_20150623_k060_m08_north.ncdf"

In [4]:
sat_df = pd.read_hdf(data_f)

In [5]:
#define the latitude and time masks
gd_t = (sat_df['DateTime'] >= dmin) & (sat_df['DateTime'] <= dmax)
gd_lat = (sat_df['SatLat'] >= lmin) & (sat_df['SatLat'] <= lmax)

In [11]:
#truncate the array so fewer aacgm 
# calculations need to be done
sat_sm = sat_df[gd_t & gd_lat].copy().reset_index()
sat_sm.shape
sat_sm.columns

(93625, 80)

Index(['index', 'DateTime', 'DateTime_omni', 'B', 'Bx_GSEGSM', 'By_GSE',
       'Bz_GSE', 'By_GSM', 'Bz_GSM', 'Vsw', 'Vx_GSE', 'Vy_GSE', 'Vz_GSE',
       'Prho', 'Tp', 'dynP', 'Esw', 'AE', 'AL', 'AU', 'SYM_D index',
       'SYM_H index', 'ASY_D index', 'ASY_H index', 'PC index',
       'Goes Proton flux (>10 MeV)', 'Goes Proton flux (>30 MeV)',
       'Goes Proton flux (>60 MeV)', 'DateTime_fism2', '225_00', '600_01',
       '1300_02', '2500_03', '5100_04', '11250_05', '18950_06', '25700_07',
       '30500_08', '43000_09', '59500_10', '72400_11', '72400_12', '85550_13',
       '85550_14', '85550_15', '94400_16', '94400_17', '94400_18', '98100_19',
       '100700_20', '103850_21', '113000_22', 'DateTime_si', 'F10', 'F81',
       'S10', 'S81c', 'M10', 'M81c', 'Y10', 'Y81c', 'DateTime_gr', 'CenterLat',
       'SatLat', 'SatLon', 'SatHeight', 'SatLT', 'SatDipoleLat', 'SatMagLon',
       'SatMagLT', 'SatDensity', '400kmDensity', '410kmDensity',
       'NRLMSISe00atSat', 'DenUncertainty', 'N

In [12]:
lat = sat_sm['SatLat'].to_numpy()
lon = sat_sm['SatLon'].to_numpy()
alt = sat_sm['SatHeight'].to_numpy()
dt = pd.to_datetime(sat_sm['DateTime_gr'].to_numpy())

x = [
    aacgmv2.get_aacgm_coord(glat,glon,galt,gdt)
    for glat, glon, galt, gdt in 
    zip(lat,lon,alt,dt)
    ]

x = np.array(x)

In [13]:
# add the new coordinates and truncate to mag coordinates
sat_sm['Sat_mlat'] = x[:,0]
sat_sm['Sat_mlon'] = x[:,1]
sat_sm['Sat_mlt'] = x[:,2]

gd_ml = (sat_sm['Sat_mlat'] >= mlmin) & (sat_sm['Sat_mlat'] <= mlmax) 
sat_sm = sat_sm[gd_ml].copy()

# calculate magnetic colatitude
sat_sm['Sat_mcolat'] = 90-sat_sm['Sat_mlat']


In [15]:
# read in the AMPERE data to get
# the AMPERE colat/mlt grid
amp_dat = nc.Dataset(data_amp)
amp_colat = amp_dat['colat'][:][0,:]
amp_mlt = amp_dat['mlt'][:][0,:]

amp_cos = np.cos(amp_mlt*np.pi/12)
amp_sin = np.sin(amp_mlt*np.pi/12)

In [17]:
pos = [(np.power(amp_cos-np.cos(x_mlt*np.pi/12),2)+
        np.power(amp_sin-np.sin(x_mlt*np.pi/12),2)+
        +np.power(amp_colat-x_lat,2)).argmin() 
        for x_mlt, x_lat in zip(sat_sm['Sat_mlt'],sat_sm['Sat_mcolat'])]


In [18]:
sat_sm['Diff_colat'] = sat_sm['Sat_mcolat']-amp_colat[pos]
sat_sm['Diff_mlt'] = sat_sm['Sat_mlt']-amp_mlt[pos] 
bd = sat_sm['Diff_mlt'] > 23.5
sat_sm.loc[bd,'Diff_mlt'] = 24-sat_sm.loc[bd,'Diff_mlt'] 

In [19]:
sat_sm['AMP_idx'] = pos

In [20]:
sat_sm

,index,DateTime,DateTime_omni,B,Bx_GSEGSM,By_GSE,Bz_GSE,By_GSM,Bz_GSM,Vsw,...,AveDragCoef,storm,storm phase,Sat_mlat,Sat_mlon,Sat_mlt,Sat_mcolat,Diff_colat,Diff_mlt,AMP_idx
0,675366,2009-01-01 00:30:00,2009-01-01 00:30:00,4.59,-4.21,1.32,-1.06,1.55,-0.68,502.7,...,2.993,-1,-1,54.366397,-7.742894,18.799389,35.633603,-0.366397,-0.200611,985
1,675367,2009-01-01 00:35:00,2009-01-01 00:35:00,4.35,-3.78,1.29,-0.86,1.46,-0.51,509.8,...,3.141,-1,-1,71.509874,-7.789085,18.880697,18.490126,0.490126,-0.119303,967
2,675368,2009-01-01 00:40:00,2009-01-01 00:40:00,4.23,-3.29,2.42,0.08,2.32,0.69,513.3,...,3.327,-1,-1,87.527427,4.395696,19.775402,2.472573,0.472573,-0.224598,1001
3,675369,2009-01-01 00:45:00,2009-01-01 00:45:00,4.24,-3.30,2.59,0.10,2.48,0.75,512.3,...,3.422,-1,-1,74.418983,164.238132,6.525884,15.581017,-0.418983,-0.474116,365
4,675370,2009-01-01 00:50:00,2009-01-01 00:50:00,4.26,-3.19,2.35,1.26,1.96,1.81,513.9,...,3.317,-1,-1,56.534604,164.029005,6.601404,33.465396,0.465396,-0.398596,382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93620,1043132,2012-06-30 23:40:00,2012-06-30 23:40:00,7.90,4.41,-5.95,-1.11,-5.79,-1.74,651.0,...,3.003,-1,-1,43.184794,21.156611,20.415427,46.815206,-0.184794,0.415427,1046
93621,1043133,2012-06-30 23:45:00,2012-06-30 23:45:00,7.90,5.04,-4.80,0.73,-4.85,0.21,651.3,...,3.154,-1,-1,62.872506,23.124866,20.629895,27.127494,0.127494,-0.370105,1076
93622,1043134,2012-06-30 23:50:00,2012-06-30 23:50:00,7.38,3.40,-4.74,0.27,-4.75,-0.22,644.8,...,3.355,-1,-1,81.388902,37.498679,21.670449,8.611098,-0.388902,-0.329551,1108
93623,1043135,2012-06-30 23:55:00,2012-06-30 23:55:00,7.11,2.11,-0.79,4.28,-1.23,4.17,656.4,...,3.452,-1,-1,80.380401,172.474658,6.745197,9.619599,-0.380401,-0.254803,359


In [22]:
x =1046
amp_colat[x]
amp_mlt[x]

47.0

20.0